In [1]:
import random
import numpy as np
# import openai
import pandas as pd
import os
import sys
import time
from scipy.sparse import csr_matrix
# Add the path to the constants file to the system path
sys.path.append('../../../')
from constants import *
from evaluation_utils import *
from path_utils import *
# from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
# openai.api_key = OPENAI_API_KEY

# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, '../data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'ml-1m/merged_data.dat')
print(f'Data path: {data_path}')

# output

CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user.dat')
print(f'Data path: {CF_OUTPUT_PATH}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user.dat')
print(f'Data path: {CF_RERUN_PATH}')

CF_OUTPUT_TIMESTAMP_PATH = os.path.join(DATA_DIR, 'ml-1m/output/split_timestamp/timestamp_large_CF_fewshot_output_path_ratings_per_user.csv')
print(f'Data path: {CF_OUTPUT_TIMESTAMP_PATH}')

CF_RERUN_TIMESTAMP_PATH = os.path.join(DATA_DIR, 'ml-1m/output/split_timestamp/rerun_timestamp_large_CF_fewshot_output_path_ratings_per_user.csv')
print(f'Data path: {CF_RERUN_TIMESTAMP_PATH}')

# Constants for column names
USER_COLUMN_NAME = 'UserID'
TITLE_COLUMN_NAME = 'Title'
ITEM_ID_COLUMN = 'MovieID'
RATING_COLUMN_NAME = 'Rating'
TIME_STAMP_COLUMN_NAME = 'Timestamp'
SYSTEM_CONTENT = MOVIELENS_CONTENT_SYSTEM


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/merged_data.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/output/CF_fewshot_output_path_ratings_per_user.dat
Data pat

In [2]:
data = pd.read_csv(data_path)
data.head(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama


In [3]:
# Create User-Item Interaction Matrix
interaction_matrix = pd.pivot_table(data, index=USER_COLUMN_NAME, columns=ITEM_ID_COLUMN, values=RATING_COLUMN_NAME).fillna(0)
csr_interaction_matrix = csr_matrix(interaction_matrix.values)

interaction_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
csr_interaction_matrix

<6040x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [7]:
# Compute the user-user Pearson Correlation Coefficient Matrix
user_pcc_matrix = pearson_correlation(csr_interaction_matrix)
print(f'User PCC Matrix:\n{user_pcc_matrix}\n')



User PCC Matrix:
[[ 1.          0.41666667 -0.33218192 ...  0.          0.05685735
  -0.04351941]
 [ 0.41666667  1.          0.23683386 ... -0.5         0.57207755
  -0.0271435 ]
 [-0.33218192  0.23683386  1.         ...  0.5         0.30927686
  -0.39528471]
 ...
 [ 0.         -0.5         0.5        ...  1.          0.27116307
  -0.39712226]
 [ 0.05685735  0.57207755  0.30927686 ...  0.27116307  1.
   0.24230884]
 [-0.04351941 -0.0271435  -0.39528471 ... -0.39712226  0.24230884
   1.        ]]



In [9]:
# Compute the item-item Pearson Correlation Coefficient Matrix
# Assuming the function 'item_pearson_correlation' takes a dense matrix as input.
# If it still takes a csr_matrix, then convert it inside the function.
dense_interaction_matrix = csr_interaction_matrix.toarray()
item_pcc_matrix = item_pearson_correlation(dense_interaction_matrix.T)
print(f'Item PCC Matrix:\n{item_pcc_matrix}\n')


Starting item-item Pearson Correlation computation...


In [ ]:

# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)
# Total number of rows with non-float ratings: 203
#Rows with non-float ratings:

Original Data Types:
user_id              int64
item_id              int64
title               object
actual_rating        int64
predicted_rating    object
dtype: object


Total number of rows with non-float ratings: 203
Rerun indices: [128, 239, 925, 1031, 1729, 2345, 3307, 4891, 5303, 5431, 5433, 5434, 5435, 5438, 5440, 5442, 5446, 5447, 5448, 5454, 5455, 5457, 5458, 5461, 5462, 5463, 5464, 5465, 5466, 5467, 5468, 5469, 5470, 5471, 5472, 5473, 5474, 5475, 5476, 5477, 5478, 5479, 5480, 5481, 5482, 5483, 5484, 5485, 5486, 5487, 5488, 5489, 5490, 5491, 5492, 5493, 5494, 5495, 5496, 5497, 5498, 5499, 5500, 5501, 5502, 5503, 5504, 5505, 5506, 5507, 5508, 5509, 5510, 5511, 5512, 5513, 5514, 5515, 5516, 5517, 5518, 5519, 5520, 5521, 5522, 5523, 5524, 5525, 5526, 5527, 5528, 5529, 5530, 5531, 5532, 5533, 5535, 5538, 5540, 5541, 5545, 5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561, 5562, 5563, 5564, 5565, 5566, 5567, 5568, 5569, 5570, 5571, 5572

,user_id,item_id,title,actual_rating,predicted_rating,is_rating_float
128,453,1254,"Treasure of the Sierra Madre, The (1948)",5,"(None, ""Request timed out: HTTPSConnectionPool...",False
239,843,3671,Blazing Saddles (1974),5,"(None, ""Request timed out: HTTPSConnectionPool...",False
925,3371,541,Blade Runner (1982),3,"(None, 'The server is overloaded or not ready ...",False


In [11]:
# def rerun_failed_CF_item_PCC_predictions(data, user_pcc_matrix, item_pcc_matrix,
#                                          save_path, user_column_name, movie_column_name,
#                                          movie_id_column, rating_column_name,
#                                          num_ratings_per_user, num_main_user_ratings, num_similar_users,
#                                          new_path, rerun_indices, seed=RANDOM_STATE,
#                                          system_content=AMAZON_CONTENT_SYSTEM):
#     # Load the original predictions
#     original_data = pd.read_csv(save_path)
#     original_data.columns = ['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating']

#     # Re-seed for reproducibility
#     random.seed(seed)

#     # Map unique users and items to their indices for quick access
#     unique_users = data[user_column_name].unique()
#     unique_items = data[movie_id_column].unique()
#     user_id_to_index = {user_id: idx for idx, user_id in enumerate(unique_users)}
#     item_id_to_index = {item_id: idx for idx, item_id in enumerate(unique_items)}

#     for index in rerun_indices:
#         user_id = original_data.at[index, 'user_id']
#         item_id = original_data.at[index, 'item_id']
#         user_idx = user_id_to_index.get(user_id)
#         item_idx = item_id_to_index.get(item_id)

#         if user_idx is None or item_idx is None:
#             print(f"User ID: {user_id} or Item ID: {item_id} not found in index. Skipping.")
#             continue

#         print(f"Rerunning prediction for User ID: {user_id}, Item ID: {item_id} (Index: {index})")

#         # Retrieve user's and item's data
#         user_data = data[data[user_column_name] == user_id]
#         item_data = data[data[movie_id_column] == item_id]

#         if item_data.empty:
#             print(f"Item data for ID: {item_id} not found. Skipping.")
#             continue

#         # Sample user's historical ratings
#         if len(user_data) < num_main_user_ratings:
#             main_user_ratings = user_data
#         else:
#             main_user_ratings = user_data.sample(n=num_main_user_ratings, random_state=seed)

#         # Construct the context from the user's ratings
#         main_user_ratings_str = '\n'.join([
#             f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars"
#             for _, row in main_user_ratings.iterrows()
#         ])

#         # Identify similar users and items
#         similar_users_idx = np.argsort(-user_pcc_matrix[user_idx])[:num_similar_users + 1]
#         similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]

#         similar_items_idx = np.argsort(-item_pcc_matrix[item_idx])[:num_similar_users + 1]
#         similar_items_idx = similar_items_idx[similar_items_idx != item_idx][:num_similar_users]

#         # Compile ratings from similar users and items
#         similar_users_ratings = ""
#         for idx in similar_users_idx:
#             similar_user_id = unique_users[idx]
#             similar_user_data = data[data[user_column_name] == similar_user_id]
#             for _, row in similar_user_data.iterrows():
#                 similar_users_ratings += f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars\n"

#         # Predict the rating
#         combined_text = f"Title: {item_data.iloc[0][movie_column_name]}"
#         prompt = f"Main User Ratings:\n{main_user_ratings_str}\n\nSimilar Users' Ratings:\n{similar_users_ratings}\n\nPredict rating for '{combined_text}':"
#         predicted_rating = predict_rating_combined_ChatCompletion(
#             combined_text, approach="CF", similar_users_ratings=similar_users_ratings,
#             rating_history=main_user_ratings_str, system_content=system_content
#         )

#         # Update the original data with the new prediction
#         original_data.at[index, 'predicted_rating'] = predicted_rating
#         print(f"Updated prediction for User ID: {user_id}, Item ID: {item_id}: {predicted_rating}")

#     # Save the updated predictions to a new file
#     original_data.to_csv(new_path, index=False)
#     print(f"Updated predictions saved to {new_path}")

In [13]:
%time
rerun_failed_CF_item_PCC_predictions(
    data=data, 
    user_pcc_matrix=user_pcc_matrix, 
    item_pcc_matrix=item_pcc_matrix,
    save_path=CF_OUTPUT_PATH, 
    user_column_name=USER_COLUMN_NAME, 
    movie_column_name=TITLE_COLUMN_NAME, 
    movie_id_column=ITEM_ID_COLUMN,
    rating_column_name=RATING_COLUMN_NAME, 
    num_ratings_per_user=NUM_RATINGS_PER_USER, 
    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
    num_similar_users=NUM_SIMILAR_USERS,
    new_path=CF_RERUN_PATH, 
    rerun_indices=rerun_indices, 
    seed=RANDOM_STATE
)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.29 µs
Rerunning prediction for User ID: 453, Item ID: 1254 (Index: 128)
Constructed Prompt for CF approach:

The prompt:
**********


Here is user rating history:
* Title: Treasure of the Sierra Madre, The (1948), Rating: 5 stars
* Title: Clockwork Orange, A (1971), Rating: 5 stars
* Title: Amos & Andrew (1993), Rating: 3 stars
* Title: Monty Python and the Holy Grail (1974), Rating: 5 stars

Here is the rating history from users who are similar to this user:
* Title: One Flew Over the Cuckoo's Nest (1975), Rating: 5 stars
* Title: My Fair Lady (1964), Rating: 4 stars
* Title: Snow White and the Seven Dwarfs (1937), Rating: 5 stars
* Title: Wizard of Oz, The (1939), Rating: 5 stars
* Title: Beauty and the Beast (1991), Rating: 5 stars
* Title: Ferris Bueller's Day Off (1986), Rating: 4 stars
* Title: Sound of Music, The (1965), Rating: 5 stars
* Title: E.T. the Extra-Terrestrial (1982), Rating: 5 stars
* Title: Hercules 

In [15]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

RMSE: 1.1372 (95% CI: (1.1153, 1.1593)) ± 0.0002
MAE: 0.8247 (95% CI: (0.8050, 0.8445)) ± 0.0002

First few actual vs predicted ratings:
Actual: 3, Predicted: 4.0000
Actual: 2, Predicted: 4.0000
Actual: 3, Predicted: 4.0000
Actual: 2, Predicted: 4.0000
Actual: 5, Predicted: 4.0000


(1.137232572602917, 0.82473309602649)

# Split by Timestamp

In [ ]:

def predict_ratings_with_CF_item_PCC_and_save_sequential(data, user_pcc_matrix, item_pcc_matrix,
                                              user_column_name='reviewerID', 
                                              movie_column_name='title', 
                                              movie_id_column='asin',
                                              rating_column_name='rating', 
                                              timestamp_column_name='Timestamp', 
                                              num_ratings_per_user=NUM_RATINGS_PER_USER, 
                                              num_similar_users=NUM_SIMILAR_USERS,
                                              num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                              test_selection_method='random',
                                              similarity_threshold=0,  # Lowered threshold
                                              save_path='cf_predictions.csv', 
                                              seed=RANDOM_STATE,
                                              system_content=AMAZON_CONTENT_SYSTEM):
    results = []

    unique_users = data[user_column_name].unique()
    unique_items = data[movie_id_column].unique()

    user_id_to_index = {user_id: idx for idx, user_id in enumerate(unique_users)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(unique_items)}

    random.seed(seed)

    for user_id in unique_users:
        user_idx = user_id_to_index[user_id]
        print(f"Processing user {user_id} (Index: {user_idx})")

        main_user_data = data[data[user_column_name] == user_id]
        
        # Select test set based on the specified method
        if test_selection_method == 'random':
            test_set, remaining_data = select_test_set_for_user(main_user_data, num_tests=TEST_OBSERVATION_PER_USER, seed=seed)
        elif test_selection_method == 'sequential':
            test_set, remaining_data = sequential_train_test_split(main_user_data, time_column=timestamp_column_name)


        if test_set.empty:
            print(f"No test data available for user {user_id}.")
            continue

        for random_movie_row in test_set.itertuples():
            random_movie_title = getattr(random_movie_row, movie_column_name)
            random_movie_id = getattr(random_movie_row, movie_id_column)
            random_movie_index = item_id_to_index.get(random_movie_id)
            actual_rating = getattr(random_movie_row, rating_column_name)

            if random_movie_index is None or random_movie_index >= item_pcc_matrix.shape[0]:
                print(f"Item '{random_movie_id}' not found or out of bounds in item_pcc_matrix.")
                continue

            if len(remaining_data) < num_main_user_ratings:
                main_user_ratings = remaining_data
            else:
                main_user_ratings = remaining_data.sample(n=num_main_user_ratings, random_state=seed)
            main_user_ratings_str = '\n'.join([
                f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars"
                for _, row in main_user_ratings.iterrows()
            ])

            similar_users_idx = np.argsort(-user_pcc_matrix[user_idx])[:num_similar_users + 1]
            similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]

            similar_users_ratings = ""
            for idx in similar_users_idx:
                similar_user_id = unique_users[idx]
                similar_user_data = data[data[user_column_name] == similar_user_id]

                # Find top-rated items by this similar user, sorted by item PCC
                similar_items_indices = np.argsort(-item_pcc_matrix[random_movie_index, :])
                top_rated_items = similar_user_data[similar_user_data[movie_id_column].isin(unique_items[similar_items_indices])]

                # Extract top ratings from this user
                top_ratings = top_rated_items.nlargest(num_ratings_per_user, rating_column_name)
                for top_rating_row in top_ratings.itertuples():
                    item_id = getattr(top_rating_row, movie_id_column)
                    rating = getattr(top_rating_row, rating_column_name)
                    item_title = data.loc[data[movie_id_column] == item_id, movie_column_name].iloc[0]  # Get the title of the item
                    similar_users_ratings += f"* Title: {item_title}, Rating: {rating} stars\n"

            combined_text = f"Title: {random_movie_title}"
            prompt = f"Main User Ratings:\n{main_user_ratings_str}\n\nSimilar Users' Ratings:\n{similar_users_ratings}\n\nPredict rating for '{combined_text}':"

            predicted_rating = predict_rating_combined_ChatCompletion(
                combined_text, 
                approach="CF", 
                similar_users_ratings=similar_users_ratings,
                rating_history=main_user_ratings_str,
                system_content=system_content
            )

            results.append([user_id, random_movie_id, random_movie_title, actual_rating, predicted_rating])
            print(f"User {user_id}: Predicted rating for '{random_movie_title}' is {predicted_rating}.")

    results_df = pd.DataFrame(results, columns=['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating'])
    results_df.to_csv(save_path, index=False)
    print(f"Predictions saved to {save_path}")

    return results_df

In [ ]:
%%time

results_df = predict_ratings_with_CF_item_PCC_and_save_sequential(
    data=data, 
    user_pcc_matrix=user_pcc_matrix, 
    item_pcc_matrix=item_pcc_matrix,
    user_column_name=USER_COLUMN_NAME, 
    movie_column_name=TITLE_COLUMN_NAME, 
    timestamp_column_name=TIME_STAMP_COLUMN_NAME,
    movie_id_column=ITEM_ID_COLUMN,
    rating_column_name=RATING_COLUMN_NAME, 
    num_ratings_per_user=NUM_RATINGS_PER_USER, 
    num_similar_users=NUM_SIMILAR_USERS,
    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
    test_selection_method='sequential',
    save_path=CF_OUTPUT_TIMESTAMP_PATH, 
    seed=RANDOM_STATE
)

0

Processing user 1 (Index: 0)


IndexError: index 4866 is out of bounds for axis 0 with size 3706

In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_OUTPUT_TIMESTAMP_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/output/split_timestamp/timestamp_large_CF_fewshot_output_path_ratings_per_user.csv'